In [3]:
import requests,re
from IPython.display import display
from bis import worms
from bis import bis
from bis import tir
from bis2 import gc2

In [4]:
q_uniqueNames = "SELECT id, \
    registration->'scientificname' AS scientificname, \
    itis->'nameWOInd' AS itisNameWOInd, \
    itis->'nameWInd' AS itisNameWInd \
    FROM tir.tir \
    WHERE worms IS NULL \
    LIMIT 25"
uniqueNames  = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q="+q_uniqueNames).json()


In [13]:
for feature in uniqueNames["features"]:
    # Set up a local data structure for storage and processing
    thisRecord = {}
    
    # Set data from query results
    thisRecord["id"] = feature["properties"]["id"]
    thisRecord["scientificname_submitted"] = feature["properties"]["scientificname"]
    thisRecord["scientificname_search"] = bis.cleanScientificName(thisRecord["scientificname_submitted"])
    thisRecord["itisNameWOInd"] = feature["properties"]["itisnamewoind"]
    thisRecord["itisNameWInd"] = feature["properties"]["itisnamewind"]
    
    # Set defaults for thisRecord
    thisRecord["matchMethod"] = "Not Matched"
    thisRecord["matchString"] = thisRecord["scientificname_search"]

    # Handle the cases where there is enough interesting stuff in the scientific name string that it comes back blank from the cleaners
    if len(thisRecord["scientificname_search"]) != 0:
        try:
            wormsSearchResults = requests.get("http://www.marinespecies.org/rest/AphiaRecordsByName/"+thisRecord["scientificname_search"]+"?like=false&marine_only=false&offset=1").json()
            thisRecord["matchMethod"] = "Exact Match"
            wormsData = wormsSearchResults[0]
        except:
            try:
                wormsSearchResults = requests.get("http://www.marinespecies.org/rest/AphiaRecordsByName/"+thisRecord["scientificname_search"]+"?like=true&marine_only=false&offset=1").json()
                thisRecord["matchMethod"] = "Fuzzy Match"
                wormsData = wormsSearchResults[0]
            except:
                wormsData = 0

    thisRecord["wormsPairs"] = worms.packageWoRMSPairs(thisRecord["matchMethod"],wormsData)
    display (thisRecord)
    # print (tir.cacheToTIR(gc2.sqlAPI("DataDistillery","BCB"),thisRecord["id"],"worms",thisRecord["wormsPairs"]))


{'id': 944,
 'itisNameWInd': 'Otus flammeolus',
 'itisNameWOInd': 'Otus flammeolus',
 'matchMethod': 'Not Matched',
 'matchString': 'Otus flammeolus',
 'scientificname_search': 'Otus flammeolus',
 'scientificname_submitted': 'Otus flammeolus',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:35:35.516355","wormsMatchMethod"=>"Not Matched"'}

{'id': 649,
 'itisNameWInd': 'Catostomus',
 'itisNameWOInd': 'Catostomus',
 'matchMethod': 'Exact Match',
 'matchString': 'Catostomus',
 'scientificname_search': 'Catostomus',
 'scientificname_submitted': 'Catostomus sp.',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:35:36.787582","wormsMatchMethod"=>"Exact Match","AphiaID"=>"158705","scientificname"=>"Catostomus","status"=>"accepted","rank"=>"Genus","valid_name"=>"Catostomus","valid_AphiaID"=>"158705","kingdom"=>"Animalia","phylum"=>"Chordata","class"=>"Actinopterygii","order"=>"Cypriniformes","family"=>"Catostomidae","genus"=>"Catostomus","lsid"=>"urn:lsid:marinespecies.org:taxname:158705","isMarine"=>"0","isBrackish"=>"0","isFreshwater"=>"1","isTerrestrial"=>"None","isExtinct"=>"None","match_type"=>"exact","modified"=>"2014-11-21T14:09:38Z"'}

{'id': 1141,
 'itisNameWInd': 'Spea intermontana',
 'itisNameWOInd': 'Spea intermontana',
 'matchMethod': 'Not Matched',
 'matchString': 'Spea intermontana',
 'scientificname_search': 'Spea intermontana',
 'scientificname_submitted': 'Spea intermontana',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:35:39.294743","wormsMatchMethod"=>"Not Matched"'}

{'id': 1039,
 'itisNameWInd': None,
 'itisNameWOInd': None,
 'matchMethod': 'Not Matched',
 'matchString': 'Rana pipiens',
 'scientificname_search': 'Rana pipiens',
 'scientificname_submitted': 'Rana pipiens',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:35:41.755905","wormsMatchMethod"=>"Not Matched"'}

{'id': 865,
 'itisNameWInd': 'Leucosticte australis',
 'itisNameWOInd': 'Leucosticte australis',
 'matchMethod': 'Not Matched',
 'matchString': 'Leucosticte australis',
 'scientificname_search': 'Leucosticte australis',
 'scientificname_submitted': 'Leucosticte australis',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:35:44.238075","wormsMatchMethod"=>"Not Matched"'}

...

{'id': 959,
 'itisNameWInd': 'Pelecanus erythrorhynchos',
 'itisNameWOInd': 'Pelecanus erythrorhynchos',
 'matchMethod': 'Exact Match',
 'matchString': 'Pelecanus erythrorhynchos',
 'scientificname_search': 'Pelecanus erythrorhynchos',
 'scientificname_submitted': 'Pelecanus erythrorhynchos',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:36:19.364484","wormsMatchMethod"=>"Exact Match","AphiaID"=>"159058","scientificname"=>"Pelecanus erythrorhynchos","status"=>"accepted","rank"=>"Species","valid_name"=>"Pelecanus erythrorhynchos","valid_AphiaID"=>"159058","kingdom"=>"Animalia","phylum"=>"Chordata","class"=>"Aves","order"=>"Pelecaniformes","family"=>"Pelecanidae","genus"=>"Pelecanus","lsid"=>"urn:lsid:marinespecies.org:taxname:159058","isMarine"=>"1","isBrackish"=>"None","isFreshwater"=>"None","isTerrestrial"=>"None","isExtinct"=>"None","match_type"=>"exact","modified"=>"2011-04-27T11:01:37Z"'}

{'id': 824,
 'itisNameWInd': 'Hybognathus placitus',
 'itisNameWOInd': 'Hybognathus placitus',
 'matchMethod': 'Not Matched',
 'matchString': 'Hybognathus placitus',
 'scientificname_search': 'Hybognathus placitus',
 'scientificname_submitted': 'Hybognathus placitus',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:36:21.646972","wormsMatchMethod"=>"Not Matched"'}

{'id': 642,
 'itisNameWInd': None,
 'itisNameWOInd': None,
 'matchMethod': 'Not Matched',
 'matchString': 'Catostomus clarki',
 'scientificname_search': 'Catostomus clarki',
 'scientificname_submitted': 'Catostomus clarki',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:36:24.189483","wormsMatchMethod"=>"Not Matched"'}

{'id': 1013,
 'itisNameWInd': 'Pseudacris triseriata',
 'itisNameWOInd': 'Pseudacris triseriata',
 'matchMethod': 'Not Matched',
 'matchString': 'Pseudacris triseriata',
 'scientificname_search': 'Pseudacris triseriata',
 'scientificname_submitted': 'Pseudacris triseriata',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:36:26.145796","wormsMatchMethod"=>"Not Matched"'}

{'id': 644,
 'itisNameWInd': None,
 'itisNameWOInd': None,
 'matchMethod': 'Not Matched',
 'matchString': 'Catostomus discobolus yarrowi',
 'scientificname_search': 'Catostomus discobolus yarrowi',
 'scientificname_submitted': 'Catostomus discobolus yarrowi',
 'wormsPairs': '"cacheDate"=>"2017-06-22T15:36:28.510746","wormsMatchMethod"=>"Not Matched"'}